In [2]:
# 安裝 & 匯入所需函式庫

!pip install pandas numpy scikit-learn matplotlib seaborn
!pip install --upgrade xgboost

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# 刪除檔案

file_path = "test.csv"  # 替換成你的檔案路徑
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"檔案 {file_path} 已刪除")
else:
    print("檔案不存在")

檔案 predictions.csv 已刪除


In [ ]:
# 下載檔案

from google.colab import files
files.download("final_predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 上傳檔案

from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [4]:
# 顯示當前目錄的所有檔案

print(os.listdir("/content/"))

['.config', 'sample_data']


In [ ]:
# 確認資料

file_path = "predictions.csv" # 讀取CSV
# df = pd.read_csv(file_path)
# df.head()

,土地移轉總面積平方公尺,交易年月,建物大於1,車位數,樓層比例,是否透天,是否投資透天,屋齡缺失,屋齡,建物移轉總面積平方公尺,...,車位類別_升降機械,車位類別_坡道機械,車位類別_平面,車位類別_無,里名,區名,區名_target,里名_target,district_village_interaction,log_unit_price
0,8.95,11301,0,1,0.670000,0,0,0,16.0,90.87,...,0,0,1,0,頂埔里,Xiangshan District,59581.21802,91035.57594,5.424010e+09,50448052.0
1,11.66,11307,0,1,0.133333,0,0,0,16.6,104.99,...,0,0,1,0,頂埔里,Xiangshan District,59581.21802,91035.57594,5.424010e+09,51771164.0


In [ ]:
# 訓練模型

# 1️⃣ 載入數據
df = pd.read_csv("test.csv")  # 訓練集

# 2️⃣ 進行 Target Encoding（使用區 & 里 的均價）
df["district_mean_price"] = df.groupby("區名")["總價元"].transform("mean")
df["village_mean_price"] = df.groupby("里名")["總價元"].transform("mean")

# 3️⃣ 標準化
scaler = StandardScaler()
df[["里名_target", "區名_target"]] = scaler.fit_transform(df[["village_mean_price", "district_mean_price"]])

# 4️⃣ 刪除不需要的欄位
df.drop(columns=["區名", "里名", "district_mean_price", "village_mean_price"], inplace=True)

# 5️⃣ 建立交叉變數、轉換成log單價
df["district_village_interaction"] = df["里名_target"] * df["區名_target"]
df["log_unit_price"] = np.log1p(df["單價"])  # Log 變換單價
df.drop(columns=["單價"], inplace=True)

# 6️⃣ 設定 X 與 y
X = df.drop(columns=["總價元"])  # 獨立變數
y = df["總價元"]  # 目標變數

# 7️⃣ 轉換成 XGBoost DMatrix
dtrain = xgb.DMatrix(X, label=y)

# 8️⃣ 設定 XGBoost 最佳參數
params = {
    "objective": "reg:squarederror",
    "eval_metric": "mae",
    "colsample_bytree": 1.0,
    "learning_rate": 0.0185,
    "max_depth": 7,
    "subsample": 0.5,
    "min_child_weight": 1,
    "reg_lambda": 0.0021,
    "reg_alpha": 0
}

# 9️⃣ 進行 5-Fold 交叉驗證（驗證最終模型表現）
cv_results = xgb.cv(params, dtrain, num_boost_round=500, nfold=5, early_stopping_rounds=50, metrics="mae", seed=42)

# 🔟 取得最小的 MAE
best_mae = cv_results["test-mae-mean"].min()
print(f"\n🎯 交叉驗證 MAE: {best_mae:,.0f}")

# 1️⃣1️⃣ 使用所有數據訓練最終模型
num_boost_round = len(cv_results)  # 取得最佳迭代次數
final_model = xgb.train(params, dtrain, num_boost_round=num_boost_round)

# 1️⃣2️⃣ 儲存模型
final_model.save_model("xgboost_final_model.json")
print("\n✅ 最終模型已儲存為 xgboost_final_model.json")


🎯 交叉驗證 MAE: 1,006,454

✅ 最終模型已儲存為 xgboost_final_model.json


In [ ]:
# 生成地區均價表

# 載入資料
df = pd.read_csv("test.csv")

# 確保沒有遺漏值
df = df.dropna(subset=["latitude", "longitude", "mean_price_per_unit", "區名", "里名"])

# 計算每個區域的平均單價
price_lookup_table = df.groupby(["區名", "里名"])["mean_price_per_unit"].mean().reset_index()
price_lookup_table.rename(columns={"mean_price_per_unit": "average_unit_price"}, inplace=True)

# 保存查詢表
price_lookup_table.to_csv("price_lookup_table.csv", index=False)

In [ ]:
# 模型API

# 定義函數：檢查是否為數值
def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# 定義函數：檢查是否為二元數 (0 或 1)
def is_binary(value):
    return value in [0, 1]

# 定義函數：檢查是否為指定範圍內的整數
def is_in_range(value, min_val, max_val):
    try:
        return min_val <= int(value) <= max_val
    except ValueError:
        return False

# 定義函數：自動填入區名
def get_district_name(district_code):
    district_mapping = {
        1: "East District",
        2: "North District",
        3: "Xiangshan District"
    }
    return district_mapping.get(district_code, None)

# 定義函數：One-Hot Encoding
def get_one_hot(value, categories):
    return [1 if value == cat else 0 for cat in categories]

# 定義函數：Target Encoding
def apply_target_encoding(value, column, target, train_data):
    median_encoding = train_data.groupby(column)[target].median()
    default_value = median_encoding.median()  # 設定預設值
    return median_encoding.get(value, default_value)

# 讀取訓練資料，用於 Target Encoding
train_data = pd.read_csv("test.csv")

# 輸入房屋資訊
data = {}

# 1. 土地移轉總面積平方公尺
while True:
    value = input("請輸入土地移轉總面積平方公尺：")
    if is_numeric(value):
        data["土地移轉總面積平方公尺"] = float(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入數值！")

# 2. 交易年月
while True:
    value = input("請輸入交易年月（範例：11403 表示民國114年3月）：")
    if value.isdigit() and len(value) == 5:
        data["交易年月"] = int(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入5位數的民國年月份（例如11403）！")

# 3. 建物是否大於1
while True:
    value = input("建物是否大於1 (0=否, 1=是)：")
    if is_binary(int(value)):
        data["建物大於1"] = int(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入0或1！")

# 4. 車位數
while True:
    value = input("請輸入車位數：")
    if value.isdigit():
        data["車位數"] = int(value)
        if data["車位數"] == 0:
            data["車位總價元"] = 0  # 若車位數為0，車位總價元自動填0
        break
    else:
        print("⚠️ 輸入錯誤：請輸入整數！")

# 5. 樓層比例
while True:
    value = input("請輸入樓層比例 (0~1，例如10/15=0.67)：")
    if is_numeric(value) and 0 <= float(value) <= 1:
        data["樓層比例"] = float(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入0到1之間的數值！")

# 6. 是否為透天
while True:
    value = input("是否為透天 (0=否, 1=是)：")
    if is_binary(int(value)):
        data["是否透天"] = int(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入0或1！")

# 7. 是否為投資透天
while True:
    value = input("是否為投資透天 (0=否, 1=是)：")
    if is_binary(int(value)):
        data["是否投資透天"] = int(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入0或1！")

# 8. 屋齡是否缺失
while True:
    value = input("屋齡是否缺失 (0=否, 1=是)：")
    if is_binary(int(value)):
        data["屋齡缺失"] = int(value)
        if data["屋齡缺失"] == 1:
            data["屋齡"] = 0  # 若屋齡缺失為0，屋齡自動填0
        break
    else:
        print("⚠️ 輸入錯誤：請輸入0或1！")

# 9. 屋齡
if data["屋齡缺失"] == 0:
    while True:
        value = input("請輸入屋齡：")
        if is_numeric(value):
            data["屋齡"] = float(value)
            break
        else:
            print("⚠️ 輸入錯誤：請輸入數值！")

# 10. 建物移轉總面積平方公尺
while True:
    value = input("請輸入建物移轉總面積平方公尺：")
    if is_numeric(value):
        data["建物移轉總面積平方公尺"] = float(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入數值！")

# 11. 是否毛胚
while True:
    value = input("是否毛胚 (0=否, 1=是)：")
    if is_binary(int(value)):
        data["是否毛胚"] = int(value)
        if data["是否毛胚"] == 1:
            data["建物現況格局-房"] = 0  # 若是否毛胚為0，建物房間數自動填0
        break
    else:
        print("⚠️ 輸入錯誤：請輸入0或1！")

# 12. 建物房間數
if data["是否毛胚"] == 0:
    while True:
        value = input("請輸入建物房間數：")
        if value.isdigit():
            data["建物現況格局-房"] = int(value)
            break
        else:
            print("⚠️ 輸入錯誤：請輸入整數！")

# 13. 是否有管理組織
while True:
    value = input("是否有管理組織 (0=否, 1=是)：")
    if is_binary(int(value)):
        data["有無管理組織"] = int(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入0或1！")

# 14. 車位總價元
if data["車位數"] > 0:
    while True:
        value = input("請輸入車位總價元：")
        if is_numeric(value):
            data["車位總價元"] = float(value)
            break
        else:
            print("⚠️ 輸入錯誤：請輸入數值！")

# 15. 是否為特殊交易
while True:
    value = input("是否為特殊交易 (0=否, 1=是)：")
    if is_binary(int(value)):
        data["特殊交易"] = int(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入0或1！")

# 16. 是否有電梯
while True:
    value = input("是否有電梯 (0=否, 1=是)：")
    if is_binary(int(value)):
        data["電梯"] = int(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入0或1！")

# 17. 緯度
while True:
    value = input("請輸入緯度（備註：Google查詢地址時，網址欄會顯示）：")
    if is_numeric(value):
        data["latitude"] = float(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入數值！")

# 18. 經度
while True:
    value = input("請輸入經度（備註：Google查詢地址時，網址欄會顯示）：")
    if is_numeric(value):
        data["longitude"] = float(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入數值！")

# 19. 與竹科的距離
while True:
    value = input("請輸入與竹科的距離 (km)：")
    if is_numeric(value):
        data["與竹科距離_km"] = float(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入數值！")

# 20. 是否為高裝修房
while True:
    value = input("是否為高裝修房 (0=否, 1=是)：")
    if is_binary(int(value)):
        data["high_price_decorated"] = int(value)
        break
    else:
        print("⚠️ 輸入錯誤：請輸入0或1！")

# 21. 都市土地使用分區
zone_options = ["住", "其", "商", "工", "農", "非"]
while True:
    value = input("請輸入都市土地使用分區：(1)住 (2)其 (3)商 (4)工 (5)農 (6)非：")
    if is_in_range(value, 1, 6):
        data.update(dict(zip([f"都市土地使用分區_{z}" for z in zone_options], get_one_hot(zone_options[int(value) - 1], zone_options))))
        break
    else:
        print("⚠️ 輸入錯誤：請輸入1到6之間的整數！")

# 22. 建物型態
building_types = ["公寓", "大樓", "華廈", "透天厝"]
while True:
    value = input("請輸入建物型態：(1)公寓 (2)大樓 (3)華廈 (4)透天厝：")
    if is_in_range(value, 1, 4):
        data.update(dict(zip([f"建物型態_{b}" for b in building_types], get_one_hot(building_types[int(value) - 1], building_types))))
        break
    else:
        print("⚠️ 輸入錯誤：請輸入1到4之間的整數！")

# 23. 主要用途
usage_types = ["住商用", "住家用", "其他", "商業用", "商辦用", "工業用", "辦公用", "農業用"]
while True:
    value = input("請輸入主要用途：(1)住商用 (2)住家用 (3)其他 (4)商業用 (5)商辦用 (6)工業用 (7)辦公用 (8)農業用：")
    if is_in_range(value, 1, 8):
        data.update(dict(zip([f"主要用途_{u}" for u in usage_types], get_one_hot(usage_types[int(value) - 1], usage_types))))
        break
    else:
        print("⚠️ 輸入錯誤：請輸入1到8之間的整數！")

# 24. 車位類別
parking_types = ["其他", "升降平面", "升降機械", "坡道機械", "平面", "無"]
while True:
    value = input("請輸入車位類別：(1)其他 (2)升降平面 (3)升降機械 (4)坡道機械 (5)平面 (6)無：")
    if is_in_range(value, 1, 6):
        data.update(dict(zip([f"車位類別_{p}" for p in parking_types], get_one_hot(parking_types[int(value) - 1], parking_types))))
        break
    else:
        print("⚠️ 輸入錯誤：請輸入1到6之間的整數！")

# 25. 區名
while True:
    value = input("請輸入區名（1=東區, 2=北區, 3=香山區）：")
    if is_in_range(value, 1, 3):
        data["區名"] = get_district_name(int(value))
        break
    else:
        print("⚠️ 輸入錯誤：請輸入1到3之間的整數！")

# 26. 里名
data["里名"] = input("請輸入里名：")

# 計算區名和里名的 Target Encoding
data["區名_target"] = apply_target_encoding(data["區名"], "區名", "mean_price_per_unit", train_data)
data["里名_target"] = apply_target_encoding(data["里名"], "里名", "mean_price_per_unit", train_data)

# 將輸入的資料轉換為 DataFrame
input_df = pd.DataFrame([data])

# 計算 district_village_interaction
input_df["district_village_interaction"] = input_df["區名_target"] * input_df["里名_target"]

# 載入單價查詢表
price_lookup_table = pd.read_csv("price_lookup_table.csv")

# 將查詢表與輸入資料合併
input_df = pd.merge(input_df, price_lookup_table, on=["區名", "里名"], how="left")

# 如果某些區域沒有對應的平均單價，使用全局平均單價填充
global_average_unit_price = price_lookup_table["average_unit_price"].mean()
input_df["average_unit_price"].fillna(global_average_unit_price, inplace=True)

# 計算 log_unit_price
input_df["log_unit_price"] = np.log1p(input_df["average_unit_price"])

# 計算預測總價
input_df["predicted_total_price"] = input_df["average_unit_price"] * input_df["建物移轉總面積平方公尺"]

# 輸出最終預測結果
print("\n🎯 預測結果：")
print(f"平均單價: {input_df['average_unit_price'].values[0]:,.2f} 元/平方公尺")
print(f"預測總價: {input_df['predicted_total_price'].values[0]:,.0f} 元")

請輸入土地移轉總面積平方公尺：21.22
請輸入交易年月（範例：11403 表示民國114年3月）：11402
建物是否大於1 (0=否, 1=是)：0
請輸入車位數：1
請輸入樓層比例 (0~1，例如10/15=0.67)：0.625
是否為透天 (0=否, 1=是)：0
是否為投資透天 (0=否, 1=是)：0
屋齡是否缺失 (0=否, 1=是)：0
請輸入屋齡：31
請輸入建物移轉總面積平方公尺：120.925
是否毛胚 (0=否, 1=是)：0
請輸入建物房間數：2
是否有管理組織 (0=否, 1=是)：1
請輸入車位總價元：1100000
是否為特殊交易 (0=否, 1=是)：0
是否有電梯 (0=否, 1=是)：1
請輸入緯度（備註：Google查詢地址時，網址欄會顯示）：24.8183626
請輸入經度（備註：Google查詢地址時，網址欄會顯示）：120.9736354
請輸入與竹科的距離 (km)：4.83
是否為高裝修房 (0=否, 1=是)：0
請輸入都市土地使用分區：(1)住 (2)其 (3)商 (4)工 (5)農 (6)非：1
請輸入建物型態：(1)公寓 (2)大樓 (3)華廈 (4)透天厝：2
請輸入主要用途：(1)住商用 (2)住家用 (3)其他 (4)商業用 (5)商辦用 (6)工業用 (7)辦公用 (8)農業用：2
請輸入車位類別：(1)其他 (2)升降平面 (3)升降機械 (4)坡道機械 (5)平面 (6)無：5
請輸入區名（1=東區, 2=北區, 3=香山區）：2
請輸入里名：金華里

🎯 預測結果：
平均單價: 109,942.54 元/平方公尺
預測總價: 13,294,801 元


<ipython-input-16-794ea53437c6>:307: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  input_df["average_unit_price"].fillna(global_average_unit_price, inplace=True)


In [5]:
git add your_notebook.ipynb
git commit -m "Add Jupyter Notebook"
git push origin main

SyntaxError: invalid syntax (<ipython-input-5-7d15399bd841>, line 1)